**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions_mario import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

**Define initial variables**
- software : path to the software directory cloned from the git repository
- protein_file : path to the file containing the prepared protein/receptor (.pdb format)
- ref_file : path to the file containing the reference ligand used for binding pocket definition (.mol2 format)
- docking_library : path to the file containing the compounds to be used for docking (.sdf format)
- id_column : name of the column containing a unique identifier for each compound in the docking_library file (string)
- n_poses : number of to generate for each compound
- exhaustiveness : exhaustiveness of the docking algorithm (GNINA and SMINA only)

The software will then create a temporary directory to store the output of the various functions.

In [ ]:
software = '/home/mario/CADD22/software'
protein_file = '/home/mario/CADD22/cox1_test/receptor_protoss_prepared.pdb'
ref_file = '/home/mario/CADD22/cox1_test/crystal_ligand.mol2'
docking_library = '/home/mario/CADD22/cox1_test/Selection_of_FCHGroup_LeadLike.sdf'
id_column = 'ID'
n_poses = 10
exhaustiveness = 8

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

**Library preparation**

This function will first standardize the library using the ChemBL structure pipeline. This will remove salts and make the library consistent.
Then a single protonation state is predicted using pka_solver. This step may take a while depending on the number of compounds.
Finally, one conformer is generated per molecule using GypsumDL.
The final_library is then written to a file in the main directory (final_library.sdf)

In [ ]:
cleaned_pkasolver_df = prepare_library_pkasolver_GypsumDL_simplified(docking_library, id_column, software)

In [ ]:
test=PandasTools.LoadSDF("/home/mario/CADD22/wocondock_refactored_chatgpt/temp/final_library.sdf", idName="ID")

**Docking**

This function will dock all compounds in the receptor, using the reference ligand as a way to define the binding site. The docking results are written to the temporary folder. 

In [ ]:
all_poses = docking(protein_file, ref_file, software, exhaustiveness, n_poses)

In [ ]:
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, embedProps=True, removeHs=True, strictParsing=True)

**Clustering**

We will first load all the poses generated from the docking run. The cluster() function performs the calculation of the clustering metrics (for now simpleRMSD and electroshape similarity), then performs the clustering using the k-medoids clustering algorithm with the number of clusters optimised using silhouette score. Finally, all cluster centers are collected and written to a file in the temporary directory (/temp/clustering/) (one file per clustering metric).

In [1]:

#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions_mario import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os
software = '/home/mario/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_test_1k/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_test_1k/crystal_ligand.mol2'
docking_library = '/home/tony/CADD22/wocondock_test_1k/merged_actives_decoys.sdf'
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[16:01:31] Initializing Normalizer


The working directory has been set to: /home/tony/CADD22/wocondock_test_1k
The folder: /home/tony/CADD22/wocondock_test_1k/temp already exists


In [2]:
def simpleRMSD_calc(arg1,arg2):
# MCS identification between reference pose and target pose
    r=rdFMCS.FindMCS([arg1,arg2])
# Atom map for reference and target              
    a=arg1.GetSubstructMatch(Chem.MolFromSmarts(r.smartsString))
    b=arg2.GetSubstructMatch(Chem.MolFromSmarts(r.smartsString))
# Atom map generation     
    amap=list(zip(a,b))
# distance calculation per atom pair
    distances=[]
    for atomA, atomB in amap:
        pos_A=arg1.GetConformer().GetAtomPosition (atomA)
        pos_B=arg2.GetConformer().GetAtomPosition (atomB)
        coord_A=np.array((pos_A.x,pos_A.y,pos_A.z))
        coord_B=np.array ((pos_B.x,pos_B.y,pos_B.z))
        dist_numpy = np.linalg.norm(coord_A-coord_B)        
        distances.append(dist_numpy)      
# This is the RMSD formula from wikipedia
    rmsd=math.sqrt(1/len(distances)*sum([i*i for i in distances]))
    return round(rmsd, 3)


In [3]:
import concurrent.futures
import progressbar
def matrix_calculation_and_clustering(method, df_name, protein_file): 
    methods = {'RMSD': simpleRMSD_calc, 'spyRMSD': spyRMSD_calc, 'espsim': espsim_calc, 'USRCAT': USRCAT_calc, 'SPLIF': SPLIF_calc, '3DScore': '3DScore'}
    df_name.index = range(len(df_name['Molecule']))
    table = pd.DataFrame(0.0, index=[df_name['Pose ID']], columns=df_name['Molecule'])
    if method == '3DScore':
        for subset in itertools.combinations(df_name['Molecule'], 2):
            result = methods['spyRMSD'](subset[0], subset[1])
            table.iloc[df_name[df_name['Molecule']==subset[0]].index.values, df_name[df_name['Molecule']==subset[1]].index.values] = 0 if np.isnan(result) else result
            table.iloc[df_name[df_name['Molecule']==subset[1]].index.values, df_name[df_name['Molecule']==subset[0]].index.values] = 0 if np.isnan(result) else result
        table['3DScore'] = table.sum(axis=1)
        table.sort_values(by='3DScore', ascending=True)
        table = table.head(1)
        table.reset_index(inplace=True)
        table = pd.DataFrame(table['Pose ID'])
        table['Pose ID'] = table['Pose ID'].astype(str).str.replace('[()\',]','', regex=False)
        #clustered_dataframes.append(table)
        return table
    else:
        
        for subset in itertools.combinations(df_name['Molecule'], 2):
            #print("subset[0]",str(subset[0])," subset[1]: ", str(subset[1]), "protein_file: ",str(protein_file))
            try:
                result = simpleRMSD_calc(subset[0], subset[1])
            except Exception as e:
                print("Exception in method:", e)
            table.iloc[df_name[df_name['Molecule']==subset[0]].index.values, df_name[df_name['Molecule']==subset[1]].index.values] = 0 if np.isnan(result) else result
            table.iloc[df_name[df_name['Molecule']==subset[1]].index.values, df_name[df_name['Molecule']==subset[0]].index.values] = 0 if np.isnan(result) else result
        clust_df = kmedoids_S_clustering(table)
        clust_df=clust_df['Pose ID']
        return clust_df
        #clustered_dataframes.append(clust_df)
    #full_df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['Pose ID'], how='outer'), clustered_dataframes)
    #full_df['Pose ID'] = full_df['Pose ID'].astype(str).replace('[()\',]','', regex=True)
    #return full_df
def cluster(method, w_dir, protein_file):
    #all_poses = all_poses.groupBy()
    id_list = np.unique(np.array(all_poses['ID']))
    create_clustering_folder(w_dir+'/temp/clustering/')
    clustered_dataframes = []
    print("*Calculating {} metrics and clustering*".format(method))
    methods = {'RMSD': simpleRMSD_calc, 'spyRMSD': spyRMSD_calc, 'espsim': espsim_calc, 'USRCAT': USRCAT_calc, 'SPLIF': SPLIF_calc, '3DScore': '3DScore'}
    with concurrent.futures.ProcessPoolExecutor() as executor:
        jobs = []
        numMol=0
        for current_id in id_list:
            try:
                job = executor.submit(matrix_calculation_and_clustering, methods[method], all_poses[all_poses['ID']==current_id], protein_file)
                jobs.append(job)
            except Exception as e:
                print("Error: ", str(e))
            #numMol = numMol+1
        widgets = ["Generating conformations; ", progressbar.Percentage(), " ", progressbar.ETA(), " ", progressbar.Bar()]
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(jobs))
        for job in pbar(concurrent.futures.as_completed(jobs)):		
            try:
                res = job.result()
                clustered_dataframes.append(res)
            except Exception as e:
                print("Error: ", str(e))
    full_df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['Pose ID'], how='outer'), clustered_dataframes)
    full_df['Pose ID'] = full_df['Pose ID'].astype(str).replace('[()\',]','', regex=True)
    #return full_df
    #clustered_poses = matrix_calculation_and_clustering(method, all_poses, id_list, protein_file, w_dir)
    clustered_poses = pd.merge(all_poses, full_df, on='Pose ID')
    # keep only the necessary columns
    clustered_poses = clustered_poses[['Pose ID', 'Molecule']]
    save_path = w_dir + '/temp/clustering/' + method + '_clustered.sdf'
    PandasTools.WriteSDF(clustered_poses, save_path, molColName='Molecule', idName='Pose ID')
    return clustered_poses


In [5]:
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, embedProps=True, removeHs=True, strictParsing=True)

In [6]:
cp = cluster('RMSD', w_dir, protein_file)
len(cp.index)

Clustering folder already exists
*Calculating RMSD metrics and clustering*


Generating conformations; 100% Time:  0:05:23 |###############################|


54230

In [ ]:
cluster('espsim', w_dir, protein_file)
cluster('spyRMSD', w_dir, protein_file)
cluster('USRCAT', w_dir, protein_file)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [ ]:
RMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/RMSD_clustered.sdf')
espsim_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/espsim_clustered.sdf')
spyRMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/spyRMSD_clustered.sdf')
USRCAT_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/USRCAT_clustered.sdf')


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
from scripts.ranking_functions import *
apply_ranking_methods_simplified(w_dir)

In [ ]:
test_df = pd.read_csv('/home/tony/CADD22/wocondock_refactored_chatgpt/temp/ranking/ranking_results.csv')
def show_correlation(dataframe):
    matrix = dataframe.corr().round(2)
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    sns.heatmap(matrix, mask = mask, annot=False, vmax=1, vmin=-1, center=0, linewidths=.5, cmap='coolwarm')
    plt.show()

show_correlation(test_df)

In [ ]:
#calculate_EFs(w_dir, docking_library)